In [5]:
import faiss
import json
import numpy as np
import re
from openai import OpenAI
from dotenv import load_dotenv
import os

# === Load env and OpenAI key ===
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# === Load index and metadata ===
car_index = faiss.read_index("/home/omar/rag/production_data/ar_faiss.index")
with open("car_metadata.json", "r", encoding="utf-8") as f:
    car_metadata = json.load(f)

# === Helpers ===
def embed_query(text: str) -> np.ndarray:
    """Embed a query using OpenAI."""
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response.data[0].embedding).astype("float32").reshape(1, -1)

def parse_specs(text: str) -> dict:
    """Extract structured fields from freeform car text."""
    specs = {
        "body_type": None,
        "horsepower": 0,
        "top_speed": 0
    }

    # Body type
    if match := re.search(r"(suv|sedan|coupe|hatchback|convertible)", text.lower()):
        specs["body_type"] = match.group(1)

    # Horsepower
    if match := re.search(r"(\d+)\s*hp", text.lower()):
        specs["horsepower"] = int(match.group(1))

    # Top speed
    if match := re.search(r"(\d+)\s*km/h", text.lower()):
        specs["top_speed"] = int(match.group(1))

    return specs

def car_agent(state: dict) -> dict:
    try:
        query = state["user_query"]
        query_embedding = embed_query(query)
        distances, indices = car_index.search(query_embedding, k=10)

        # Raw candidates
        candidates = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx < len(car_metadata):
                meta = car_metadata[idx]
                specs = parse_specs(meta["text"])
                candidates.append({
                    "id": meta["id"],
                    "text": meta["text"],
                    "score": float(dist),
                    **specs
                })

        # Filtering logic (coupe + >300 HP + top speed > 200)
        filtered = [
            c for c in candidates
            if c["body_type"] == "coupe" and c["horsepower"] > 300 and c["top_speed"] > 200
        ]

        # Rank by score + specs relevance (simple sum)
        def rank(car):
            # The lower the FAISS score, the more relevant.
            return -car["score"] + 0.001 * car["horsepower"] + 0.001 * car["top_speed"]

        final_results = sorted(filtered if filtered else candidates, key=rank, reverse=True)[:3]

        # Format output
        if not final_results:
            state["answer"] = "❌ No relevant coupe with high horsepower found."
        else:
            lines = []
            for car in final_results:
                lines.append(
                    f"### 🚗 {car['id']}  \n"
                    f"**Relevance**: {car['score']:.2f}  \n"
                    f"**Type**: {car['body_type']}  \n"
                    f"**Horsepower**: {car['horsepower']} HP  \n"
                    f"**Top Speed**: {car['top_speed']} km/h  \n"
                    f"{car['text']}"
                )
            state["answer"] = "\n\n---\n\n".join(lines)

    except Exception as e:
        state["answer"] = f"❌ Error in car_agent: {str(e)}"

    return state


In [6]:
state = {"user_query": "Show me a coupe with high horsepower and fast speed"}
result = car_agent(state)
print(result["answer"])


### 🚗 PartCatch65  
**Relevance**: 0.35  
**Type**: coupe  
**Horsepower**: 568 HP  
**Top Speed**: 236 km/h  
PartCatch65 by Ballard-Gonzalez (Launched in 2017).
Description: Experience the fusion of style and performance with the PartCatch65 from Ballard-Gonzalez. This agile coupe boasts unmatched reliability along with unmatched reliability. Its innovative design and reliable engineering, launched in 2017, set a new benchmark in its class.
Engine: V8, 568 HP, 2397cc
Specs: Convertible, 6 km/l, 236 km/h top speed
User Rating: 2.6/5 | NCAP Rating: 4/5

---

### 🚗 Cohen-Hood  
**Relevance**: 0.34  
**Type**: coupe  
**Horsepower**: 486 HP  
**Top Speed**: 219 km/h  
Cohen-Hood by FightQuite32 (Launched in 1999).
Description: FightQuite32 presents the Cohen-Hood, a dynamic masterpiece that redefines modern mobility. With features such as next-generation navigation system and exceptional handling, this coupe combines efficiency with bold performance. Since its inception in 1999, it has c

In [ ]:
import re

def route_query(state: dict) -> dict:
    """
    Analyze the user's query and assign a routing label in state['route'].
    Supported agents: car_agent, country_agent, math_agent, fallback
    """
    query = state.get("user_query", "").strip().lower()

    # === Car-related keywords ===
    car_keywords = [
        "car", "engine", "horsepower", "torque", "fuel", "mileage",
        "km/l", "hp", "cc", "suv", "hatchback", "sedan", "convertible",
        "model", "brand", "launch", "top speed", "vehicle", "ncap"
    ]

    # === Country-related keywords ===
    country_keywords = [
        "country", "capital", "language", "population", "area",
        "national animal", "national bird", "river", "continent",
        "borders", "official languages", "government", "flag",
        "demographics", "national symbol"
    ]

    # === Math pattern matchers ===
    math_pattern = re.compile(r"[\d\s\+\-\*/\(\)=^%!√]+")
    math_keywords = ["calculate", "solve", "result of", "math", "expression", "evaluate", "equation"]

    # === Fuzzy matching logic ===

    # Car detection
    if any(word in query for word in car_keywords):
        state["route"] = "car_agent"
        return state

    # Country detection
    elif any(word in query for word in country_keywords):
        state["route"] = "country_agent"
        return state

    # Math detection: combination of keyword and pattern
    elif any(word in query for word in math_keywords) or math_pattern.fullmatch(query.replace(" ", "")):
        state["route"] = "math_agent"
        return state

    # Fallback if nothing matches
    state["route"] = "fallback"
    return state
